## Trim raw reads using trimmomatic 
-------


In [ ]:
cd /path/to/raw/reads

for R1 in *R1*
do
   R2=${R1//R1.fastq.gz/R2.fastq.gz}
   R1paired=${R1//.fastq.gz/_paired.fastq.gz}
   R1unpaired=${R1//.fastq.gz/_unpaired.fastq.gz}
   R2paired=${R2//.fastq.gz/_paired.fastq.gz}
   R2unpaired=${R2//.fastq.gz/_unpaired.fastq.gz} \
   java -jar trimmomatic-0.39.jar PE $R1 $R2 $R1paired $R1unpaired $R2paired $R2unpaired \
   ILLUMINACLIP:TruSeq3-PE.fa:2:30:10 LEADING:5 TRAILING:5 SLIDINGWINDOW:4:15 MINLEN:100
done

## RUN FASTQC #TODOOOOOO

## Assembly with SPADES 
-----

In [ ]:
for file1 in /path/to/trimmed/data/directory/*R1*fastq
do 
file2=${file1/R1/R2}
out=${file1%%_R1.fastq}_output
spades.py -k 21,33,55,77 -c --only-assembler -1 $file1 -2 $file2 -o $out
done

## Ref Alignment 
------

In [ ]:
##John  Stuff

## SV Calling 
-----

**python script for making pindel config**

In [ ]:
import os 
for file in os.listdir("/home/sharrison/data/bams/"):
    bam = "."
    if file.endswith(".stats"):
        mySample = file.split(".")[0]
        theBam = file.split(".")
        theBam = theBam[:-1]
        print(theBam)
        bam = bam.join(theBam)
        myBam = "/home/sharrison/data/bams/" + file 
        file1 = open(myBam, "r")
        print(myBam)
        for line in file1.readlines():
            if line.startswith("SN"):
                if line.split("\t")[1] == "insert size average:":
                    insertSize = line.rstrip().split("\t")[2]
        myText = "/home/sharrison/data/bams/" + bam + "\t" + insertSize + "\t" + mySample
        newName = mySample + "_pindel.cfg"
        pFile = open(newName, "w")
        pFile.write(myText)
        pFile.close()
        file1.close()   

**Python script for making script to run lumpy**

In [ ]:
import os 
myFile = "#!/bin/bash" + "\n"
for file in os.listdir("/home/sharrison/data/bams/"):
    bam = "."
    if file.endswith(".stats"):
        mySample = file.split(".")[0]
        theBam = file.split(".")
        theBam = theBam[:-1]
        bam = bam.join(theBam)
        myBam = "/home/sharrison/data/bams/" + file 
        file1 = open(myBam, "r")
        for line in file1.readlines():
            if line.startswith("SN"):
                if line.split("\t")[1] == "insert size average:":
                    insertSize = line.rstrip().split("\t")[2]
                if line.split("\t")[1] == "insert size standard deviation:":
                    sd = line.rstrip().split("\t")[2]
                if line.split("\t")[1] == "average length:":
                    readLen = line.rstrip().split("\t")[2]
        myText = "lumpy -mw 4 -tt 0 -pe id:" + mySample + ",bam_file:" + mySample + ".discordants.bam," + "histo_file:" + mySample + ".lib1.histo,mean:" + insertSize + ",stdev:" + sd + ",read_length:" + readLen + ",min_non_overlap:" + readLen + ",discordant_z:5,back_distance:10,weight:1,min_mapping_threshold:20" + " -sr id:" + mySample + ",bam_file:" + mySample + ".splitters.bam" + ",back_distance:10,weight:1,min_mapping_threshold:20" + " > " + mySample + "_lumpy.vcf" + "\n"
        myFile += myText


pFile = open("runLumpy.sh", "w")
pFile.write(myFile)
pFile.close()

**Running all steps**

In [ ]:

##change data to whatever data folder called

cd ~/data

mkdir pindel
mkdir delly
mkdir breakdancer
mkdir manta
mkdir gridss
mkdir lumpy 

##first need to run stats on each sample to properly make config files 
cd ~/data/bams
for num in {1..22}
do
samtools stats bm{num}.bam > bm{num}.stats
done


##pindel run pindel.sh script after have cfg files
cd ~/data/pindel
python3 makePindelCfg.py 
chmod u+x pindel.sh
nohup ./pindel.sh &

###breakdancer
cd ~/data/breakdancer

for num in {1..22}
do
perl /usr/local/breakdancer/perl/bam2cfg.pl -g -h ~/data/bams/bm${num}.sorted.bam > bm${num}.cfg
breakdancer-max -r 4 bm${num}.cfg > bm${num}_breakdancer.txt
done

###delly 
cd ~/data/delly

for num in {1..22}
do
delly call -g ~/data/ref/ref246.fa -o bm${num}.bcf ~/data/bams/bm${num}.sorted.bam 
done

delly merge -o all_sites.bcf *.bcf

for num in {1..22}
do
delly call -g ~/data/ref/ref246.fa -v all_sites.bcf -o bm${num}_delly.bcf ~/data/bams/bm${num}.sorted.bam 
done

###manta
cd ~/data/manta

for num in {11..22}
do
mkdir bm${num}
configManta.py --bam ~/data/bams/bm${num}.sorted.bam --referenceFasta ~/data/ref/ref246.fa --runDir bm${num}
~/data/manta/bm${num}/runWorkflow.py
done


###gridss
cd ~/data/gridss
for num in {1..22}
do
gridss.sh --jar /usr/local/gridss/scripts/gridss.jar --reference ~/data/ref/ref246.fa --output bm${num}_gridss.vcf.gz --labels bm${num}  --assembly bm${num}_g.bam ~/data/bams/bm${num}.sorted.bam 
done


###lumpy
cd ~/data/lumpy

for num in {1..22}
do
samtools view -b -F 1294 ~/data/bams/bm${num}.sorted.bam > bm${num}.discordants.bam
samtools view -h ~/data/bams/bm${num}.sorted.bam \
    | /usr/local/lumpy-sv/scripts/extractSplitReads_BwaMem -i stdin \
    | samtools view -Sb - \
    > bm${num}.splitters.bam
done

# will probably have to change for 1,11,12 sample because not 100 for sequence reads

listS=(2 3 4 5 6 7 8 9 10 13 14 15 16 17 18 19 20 21 22)
for num in ${listS[@]}
do
samtools view ~/data/bams/bm${num}.sorted.bam \
    | tail -n+100000 \
    | /usr/local/lumpy-sv/scripts/pairend_distro.py \
    -r 250 \
    -X 4 \
    -N 10000 \
    -o bm${num}.lib1.histo
done

listS=(1 11 12)
for num in ${listS[@]}
do
samtools view ~/data/bams/bm${num}.sorted.bam \
    | tail -n+100000 \
    | /usr/local/lumpy-sv/scripts/pairend_distro.py \
    -r 100 \
    -X 4 \
    -N 10000 \
    -o bm${num}.lib1.histo
done

###run the lumpy script with commands for each sample
python3 lumpyRun.sh
chmod u+x runLumpy.sh
nohup ./runLumpy.sh &


In [ ]:
import pandas as pd
import io

def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

def reformat_breakdancer(file):
    unformat_break = pd.read_csv(file, sep='\t', skiprows=[0,1,2,3])
    info_list = []
    for index,row in unformat_break.iterrows():
        info_line = "SVLEN=" + str(row["Size"]) + ":END=" + str(row["Pos2"]) + ":num_reads=" + str(row["num_Reads"])
        info_list.append(info_line)
    new_breakdancer = pd.DataFrame(columns=['#CHROM', 'POS', 'ID', 'REF', 'ALT','QUAL', 'FILTER', 'INFO'])
    chrom_list = unformat_break['#Chr1'].to_list()
    pos_list = unformat_break['Pos1'].to_list()
    alt_list = unformat_break['Type'].to_list()
    qual_list = unformat_break['Score'].to_list()
    new_breakdancer['#CHROM'] = chrom_list
    new_breakdancer['POS'] = pos_list
    new_breakdancer['ID'] = "breakdancer"
    new_breakdancer['REF'] = "N"
    new_breakdancer['ALT'] = alt_list 
    new_breakdancer['QUAL'] = qual_list 
    new_breakdancer['FILTER'] = "."
    new_breakdancer['INFO'] = info_list
    return new_breakdancer 


def reformat_ss(manta,caller):  
    end_list = [] 
    if caller == 'breakdancer':
        for index,row in manta.iterrows():
            if row['ALT'] == 'CTX':
                manta.drop(index, inplace=True)
            else:
                end = row['INFO'].split('END=')[1].split(":")[0]
                end_list.append(end)
        chroms = manta['CHROM'].to_list()
        start_pos = manta['POS'].to_list()
        svtype_list = manta['ALT'].to_list()
        qual = manta['QUAL'].to_list()
        new_fr = pd.DataFrame(columns=['CHROM', 'START_POS', 'STOP_POS', 'SV_TYPE', 'QUAL', 'CALLER'])
        new_fr['CHROM'] = chroms
        new_fr['START_POS'] = start_pos
        new_fr['STOP_POS'] = end_list
        new_fr['SV_TYPE'] = svtype_list
        new_fr['QUAL'] = qual
        new_fr['CALLER'] = caller
        
    elif caller == 'pindel':
        for index,row in manta.iterrows():
            SV_type = row['INFO'].split('SVTYPE=')[1].split(';')[0]
            end = row['INFO'].split('END=')[1].split(";")[0]
            end_list.append(end)
            new = SV_type 
            manta.at[index,'ALT'] = new
        chroms = manta['CHROM'].to_list()
        start_pos = manta['POS'].to_list()
        svtype_list = manta['ALT'].to_list()
        new_fr = pd.DataFrame(columns=['CHROM', 'START_POS', 'STOP_POS', 'SV_TYPE', 'QUAL', 'CALLER'])
        new_fr['CHROM'] = chroms
        new_fr['START_POS'] = start_pos
        new_fr['STOP_POS'] = end_list
        new_fr['SV_TYPE'] = svtype_list
        new_fr['QUAL'] = "."
        new_fr['CALLER'] = caller
        
    elif caller == 'delly':
        ind_l = []
        for index,row in manta.iterrows():
            gt = str(manta.iloc[index,9])
            gt_l = gt.split(":")[0]
            if gt_l == '0/0':
                ind_l.append(index)
        for index,row in manta.iterrows():
            SV_type = row['INFO'].split('SVTYPE=')[1].split(';')[0]
            if SV_type == "BND":
                end_list.append("BND")
            else:
                end = row['INFO'].split('END=')[1].split(";")[0]
                end_list.append(end)
            new = SV_type
            manta.at[index,'ALT'] = new
        chroms = manta['CHROM'].to_list()
        start_pos = manta['POS'].to_list()
        svtype_list = manta['ALT'].to_list()
        qual = manta['QUAL'].to_list()
        new_fr = pd.DataFrame(columns=['CHROM', 'START_POS', 'STOP_POS', 'SV_TYPE', 'QUAL', 'CALLER'])
        new_fr['CHROM'] = chroms
        new_fr['START_POS'] = start_pos
        new_fr['STOP_POS'] = end_list
        new_fr['SV_TYPE'] = svtype_list
        new_fr['QUAL'] = qual
        new_fr['CALLER'] = caller
        for num in ind_l:
            new_fr.drop(num, inplace=True)
    else:

        for index,row in manta.iterrows():
            SV_type = row['INFO'].split('SVTYPE=')[1].split(';')[0]
            if SV_type == "BND":
                end_list.append("BND")
            else:
                end = row['INFO'].split('END=')[1].split(";")[0]
                end_list.append(end)
            new = SV_type
            manta.at[index,'ALT'] = new
        chroms = manta['CHROM'].to_list()
        start_pos = manta['POS'].to_list()
        svtype_list = manta['ALT'].to_list()
        qual = manta['QUAL'].to_list()
        new_fr = pd.DataFrame(columns=['CHROM', 'START_POS', 'STOP_POS', 'SV_TYPE', 'QUAL', 'CALLER'])
        new_fr['CHROM'] = chroms
        new_fr['START_POS'] = start_pos
        new_fr['STOP_POS'] = end_list
        new_fr['SV_TYPE'] = svtype_list
        new_fr['QUAL'] = qual
        new_fr['CALLER'] = caller
               
    return new_fr

def combine_SV(breakdancer_file,pindel_file,delly_file,manta_file,lumpy_file,gridss_file):
    breakdancer = pd.read_csv(breakdancer_file,sep='\t')
    pindel = pd.read_csv(pindel_file,sep='\t')
    lumpy = pd.read_csv(lumpy_file,sep='\t')
    delly = pd.read_csv(delly_file,sep='\t')
    manta = pd.read_csv(manta_file,sep='\t')
    gridss = pd.read_csv(gridss_file,sep='\t')
    all_types = pd.concat([breakdancer,pindel,lumpy,delly,manta,gridss], axis=0)
    return all_types

def make_merged(file, max_size, min_size, percent_threshold, caller_threshold):
    sample_SV = pd.read_csv(file,sep='\t')
    for index, row in sample_SV.iterrows():
        if row['STOP_POS'] == "BND":
            sample_SV.drop(index, inplace=True)
    for index, row in sample_SV.iterrows(): 
        sv_len = int(row['STOP_POS']) - int(row['START_POS'])
        if int(sv_len) > int(max_size) or int(sv_len) < int(min_size):
            sample_SV.drop(index, inplace=True)
    all_overlaps = []
    min_max = []
    for r1 in sample_SV.itertuples():
        pos_overlap = ""
        smallest = float(r1.START_POS)
        largest = float(r1.STOP_POS)
        for r2 in sample_SV.itertuples():  
            if r1.CHROM == r2.CHROM:
                if float(r2.START_POS) <= float(r1.START_POS) and float(r2.STOP_POS) < float(r1.STOP_POS) and float(r2.STOP_POS) > float(r1.START_POS):      
                    caller = r2.CALLER
                    start2 = float(r2.START_POS)
                    stop2 = float(r2.STOP_POS)
                    start1 = float(r1.START_POS)
                    stop1 = float(r1.STOP_POS)
                    sv_type = r2.SV_TYPE
                    overlap = (stop2 - start1) / (stop1 - start1) * 100
                    overlap_2 = (stop2 - start1) / (stop2 - start2) * 100
                    if overlap >= float(percent_threshold) and overlap_2 >= float(percent_threshold):
                        info = "ID=" + caller + ':' + "START=" + str(start2) + ':' + "END=" + str(stop2) + ':' + sv_type + ';' 
                        pos_overlap += info
                        if start2 < smallest:
                            smallest = start2
                        elif stop2 > largest:
                            largest = stop2
                elif float(r2.START_POS) > float(r1.START_POS) and float(r2.STOP_POS) >= float(r1.STOP_POS) and float(r2.START_POS) < float(r1.STOP_POS):
                    caller = r2.CALLER
                    start2 = float(r2.START_POS)
                    stop2 = float(r2.STOP_POS)
                    start1 = float(r1.START_POS)
                    stop1 = float(r1.STOP_POS)
                    sv_type = r2.SV_TYPE
                    overlap = (stop1 - start2) / (stop1 - start1) * 100
                    overlap_2 = (stop1 - start2) / (stop2 - start2) * 100
                    if overlap >= float(percent_threshold) and overlap_2 >= float(percent_threshold):
                        info = "ID=" + caller + ':' + "START=" + str(start2) + ':' + "END=" + str(stop2) + ':' + sv_type + ';' 
                        pos_overlap += info 
                        if start2 < smallest:
                            smallest = start2
                        elif stop2 > largest:
                            largest = stop2
                elif float(r2.START_POS) <= float(r1.START_POS) and float(r2.STOP_POS) >= float(r1.STOP_POS):
                    caller = r2.CALLER
                    start2 = float(r2.START_POS)
                    stop2 = float(r2.STOP_POS)
                    start1 = float(r1.START_POS)
                    stop1 = float(r1.STOP_POS)
                    sv_type = r2.SV_TYPE
                    overlap = (stop1 - start1) / (stop2 - start2) * 100
                    if overlap >= float(percent_threshold):
                        info = "ID=" + caller + ':' + "START=" + str(start2) + ':' + "END=" + str(stop2) + ':' + sv_type + ';' 
                        pos_overlap += info
                        if start2 < smallest:
                            smallest = start2
                        elif stop2 > largest:
                            largest = stop2

                elif float(r2.START_POS) >= float(r1.START_POS) and float(r2.STOP_POS) <= float(r1.STOP_POS):
                    caller = r2.CALLER
                    start2 = float(r2.START_POS)
                    stop2 = float(r2.STOP_POS)
                    start1 = float(r1.START_POS)
                    stop1 = float(r1.STOP_POS)
                    sv_type = r2.SV_TYPE
                    overlap = (stop2 - start2) / (stop1 - start1) * 100 

                    if overlap >= float(percent_threshold):
                        info = "ID=" + caller + ':' + "START=" + str(start2) + ':' + "END=" + str(stop2) + ':' + sv_type + ';' 
                        pos_overlap += info 
                        if start2 < smallest:
                            smallest = start2
                        elif stop2 > largest:
                            largest = stop2
                
        all_overlaps.append(pos_overlap)
        range_info = str(smallest) + '-' + str(largest)
        min_max.append(range_info)
    how_many = []
    for item in all_overlaps:
        each_type = item.split(';')
        del each_type[-1]
        all_types = []
        for thing in each_type:
            callerID = thing.split(':')[0]
            all_types.append(callerID)
        all_types = list(dict.fromkeys(all_types))
        total = len(all_types)  
        how_many.append(total)
    sample_SV['OVERLAPS'] = all_overlaps
    sample_SV['NUM_CALLERS'] = how_many
    sample_SV['RANGE'] = min_max
    dedup_SV = sample_SV.drop_duplicates('RANGE', keep='first')
    dedup_SV_2 = dedup_SV.drop_duplicates('RANGE', keep='first')
    high_num = dedup_SV_2.loc[dedup_SV_2['NUM_CALLERS'] >= caller_threshold]
    ranges = high_num['RANGE'].to_list()
    chroms = high_num['CHROM'].to_list()
    callers = high_num['OVERLAPS'].to_list()
    num_callers = high_num['NUM_CALLERS'].to_list()
    all_starts = []
    all_stops = []
    lengths = []
    for nums in ranges:
        start = nums.split('-')[0]
        stop = nums.split('-')[1]
        length = float(stop) - float(start)
        all_starts.append(start)
        all_stops.append(stop)
        lengths.append(length)    
    everything = {'CHROM': chroms, 'START': all_starts, 'STOP': all_stops, 'SV_CALLED': callers, 'NUM_CALLERS': num_callers, 'SV_LEN': lengths}
    new_frame = pd.DataFrame(everything)   
    return new_frame
  
def most_frequent(List): 
    return max(set(List), key = List.count)

def dedup_reformat(sample_table, over_t):

    over_t = float(over_t)
    for r1 in sample_table.itertuples():
        for r2 in sample_table.itertuples():
            if r1.Index != r2.Index:
                if r1.CHROM == r2.CHROM:
                    if float(r2.START) <= float(r1.START) and float(r2.STOP) < float(r1.STOP) and float(r2.STOP) > float(r1.START):
                        overlap = (float(r2.STOP) - float(r1.START)) / (float(r1.STOP) - float(r1.START)) * 100
                        overlap_2 = (float(r2.STOP) - float(r1.START)) / (float(r2.STOP) - float(r2.START)) * 100
                        if overlap >= over_t and overlap_2 >= over_t:
                            info1 = r1.SV_CALLED.split(';')
                            info2 = r2.SV_CALLED.split(';')
                            del info1[-1]
                            del info2[-1]
                            new_info = info1 + list(set(info2) - set(info1))
                            info_str = ';'
                            info_str = info_str.join(new_info) 
                            info_str += ';'
                            new_length = float(r1.STOP) - float(r2.START)
                            sample_table.at[r1.Index, 'SV_CALLED'] = info_str
                            sample_table.at[r1.Index, 'START'] = float(r2.START)
                            sample_table.at[r1.Index, 'STOP'] = float(r1.STOP)
                            sample_table.at[r1.Index, 'CHROM'] = r1.CHROM
                            sample_table.at[r1.Index, 'SV_LEN'] = new_length
                            sample_table.at[r1.Index, 'NUM_CALLERS'] = r1.NUM_CALLERS
                            sample_table.drop(r2.Index, inplace=True)
                    elif float(r2.START) > float(r1.START) and float(r2.STOP) >= float(r1.STOP) and float(r2.START) < float(r1.STOP):
                        overlap = (float(r1.STOP) - float(r2.START)) / (float(r1.STOP) - float(r1.START)) * 100
                        overlap_2 = (float(r1.STOP) - float(r2.START)) / (float(r2.STOP) - float(r2.START)) * 100
                        if overlap >= over_t and overlap_2 >= over_t:
                            info1 = r1.SV_CALLED.split(';')
                            info2 = r2.SV_CALLED.split(';')
                            del info1[-1]
                            del info2[-1]
                            new_info = info1 + list(set(info2) - set(info1))
                            info_str = ';'
                            info_str = info_str.join(new_info) 
                            info_str += ';'
                            new_length = float(r2.STOP) - float(r1.START)
                            sample_table.at[r1.Index, 'SV_CALLED'] = info_str
                            sample_table.at[r1.Index, 'CHROM'] = r1.CHROM
                            sample_table.at[r1.Index, 'SV_LEN'] = new_length
                            sample_table.at[r1.Index, 'NUM_CALLERS'] = r1.NUM_CALLERS
                            sample_table.at[r1.Index, 'STOP'] = float(r2.STOP)
                            sample_table.at[r1.Index, 'START'] = float(r1.START)
                            sample_table.drop(r2.Index, inplace=True)
                    elif float(r2.START) <= float(r1.START) and float(r2.STOP) >= float(r1.STOP):
                        overlap = (float(r1.STOP) - float(r1.START)) / (float(r2.STOP) - float(r2.START)) * 100
                        if overlap >= over_t:
                            info1 = r1.SV_CALLED.split(';')
                            info2 = r2.SV_CALLED.split(';')
                            del info1[-1]
                            del info2[-1]
                            new_info = info1 + list(set(info2) - set(info1))
                            info_str = ';'
                            info_str = info_str.join(new_info)
                            info_str += ';'
                            new_length = float(r2.STOP) - float(r2.START)
                            sample_table.at[r1.Index, 'SV_CALLED'] = info_str
                            sample_table.at[r1.Index, 'START'] = float(r2.START)
                            sample_table.at[r1.Index, 'STOP'] = float(r2.STOP)
                            sample_table.at[r1.Index, 'CHROM'] = r1.CHROM
                            sample_table.at[r1.Index, 'SV_LEN'] = new_length
                            sample_table.at[r1.Index, 'NUM_CALLERS'] = r1.NUM_CALLERS
                            sample_table.drop(r2.Index, inplace=True)
                    elif float(r2.START) >= float(r1.START) and float(r2.STOP) <= float(r1.STOP):
                        overlap = (float(r2.STOP) - float(r2.START)) / (float(r1.STOP) - float(r1.START)) * 100
                        if overlap >= over_t:
                            info1 = r1.SV_CALLED.split(';')
                            info2 = r2.SV_CALLED.split(';')
                            del info1[-1]
                            del info2[-1]
                            new_info = info1 + list(set(info2) - set(info1))
                            info_str = ';'
                            info_str = info_str.join(new_info) 
                            info_str += ';'
                            sample_table.at[r1.Index, 'SV_CALLED'] = info_str
                            sample_table.at[r1.Index, 'START'] = float(r1.START)
                            sample_table.at[r1.Index, 'STOP'] = float(r1.STOP)
                            sample_table.at[r1.Index, 'CHROM'] = r1.CHROM
                            sample_table.at[r1.Index, 'SV_LEN'] = r1.SV_LEN
                            sample_table.at[r1.Index, 'NUM_CALLERS'] = r1.NUM_CALLERS
                            sample_table.drop(r2.Index, inplace=True)

    breakdancer = []
    pindel = []
    lumpy = []
    manta = []
    gridss = []
    delly = []

    for row1 in sample_table.itertuples():
        br_ss = []
        br_t = []
        pi_ss = []
        pi_t = []
        lu_ss = []
        lu_t = []
        ma_ss = []
        ma_t = []
        gr_ss = []
        gr_t = []
        de_ss = []
        de_t = []
        ind = row1.SV_CALLED.split(';')
        del ind[-1]
        for thing in ind:
            each = thing.split(':')
            ID = each[0].split('=')[1]
            START = each[1].split('=')[1]
            STOP = each[2].split('=')[1]
            TYPE = each[3]
            if ID == 'breakdancer':
                br_ss.append(float(START))
                br_ss.append(float(STOP))
                br_t.append(TYPE)
            elif ID == 'pindel':
                pi_ss.append(float(START))
                pi_ss.append(float(STOP))
                pi_t.append(TYPE)
            elif ID == 'lumpy':
                lu_ss.append(float(START))
                lu_ss.append(float(STOP))
                lu_t.append(TYPE)
            elif ID == 'delly':
                de_ss.append(float(START))
                de_ss.append(float(STOP))
                de_t.append(TYPE) 
            elif ID == 'manta':
                ma_ss.append(float(START))
                ma_ss.append(float(STOP))
                ma_t.append(TYPE)
            elif ID == 'gridss':
                gr_ss.append(float(START))
                gr_ss.append(float(STOP))
                gr_t.append(TYPE)      
        if len(br_ss) != 0:
            s = float(min(br_ss))
            e = float(max(br_ss))
            percent_o = (e - s) / (float(row1.STOP) - float(row1.START)) * 100
            ninfo = str(s) + '-' + str(e) + ':' + most_frequent(br_t) + ':' + str(round(percent_o, 2)) + '%'
            breakdancer.append(ninfo)
        if len(br_t) == 0:
            x = 'X'
            breakdancer.append(x)
        if len(pi_ss) != 0:
            s = float(min(pi_ss))
            e = float(max(pi_ss))
            percent_o = (e - s) / (float(row1.STOP) - float(row1.START)) * 100
            ninfo = str(s) + '-' + str(e) + ':' + most_frequent(pi_t) + ':' + str(round(percent_o, 2)) + '%'
            pindel.append(ninfo)
        if len(pi_t) == 0:
            x = 'X'
            pindel.append(x)
        if len(lu_ss) != 0:
            s = float(min(lu_ss))
            e = float(max(lu_ss))
            percent_o = (e - s) / (float(row1.STOP) - float(row1.START)) * 100
            ninfo = str(s) + '-' + str(e) + ':' + most_frequent(lu_t) + ':' + str(round(percent_o, 2)) + '%'
            lumpy.append(ninfo)
        if len(lu_t) == 0:
            x = 'X'
            lumpy.append(x)
        if len(de_ss) != 0:
            s = float(min(de_ss))
            e = float(max(de_ss))
            percent_o = (e - s) / (float(row1.STOP) - float(row1.START)) * 100
            ninfo = str(s) + '-' + str(e) + ':' + most_frequent(de_t) + ':' + str(round(percent_o, 2)) + '%'
            delly.append(ninfo)
        if len(de_t) == 0:
            x = 'X'
            delly.append(x)
        if len(ma_ss) != 0:
            s = float(min(ma_ss))
            e = float(max(ma_ss))
            percent_o = (e - s) / (float(row1.STOP) - float(row1.START)) * 100
            ninfo = str(s) + '-' + str(e) + ':' + most_frequent(ma_t) + ':' + str(round(percent_o, 2)) + '%'
            manta.append(ninfo)
        if len(ma_t) == 0:
            x = 'X'
            manta.append(x)
        if len(gr_ss) != 0:
            s = float(min(gr_ss))
            e = float(max(gr_ss))
            percent_o = (e - s) / (float(row1.STOP) - float(row1.START)) * 100
            ninfo = str(s) + '-' + str(e) + ':' + most_frequent(gr_t) + ':' + str(round(percent_o, 2)) + '%'
            gridss.append(ninfo)
        if len(gr_t) == 0:
            x = 'X'
            gridss.append(x)
    starts = sample_table['START'].to_list()
    stops = sample_table['STOP'].to_list()
    chroms = sample_table['CHROM'].to_list()
    num_callers = sample_table['NUM_CALLERS'].to_list()
    all_lengths = sample_table['SV_LEN'].to_list()
    all_reformat = {'CHROM': chroms, 'START': starts, 'STOP': stops,'NUM_CALLERS': num_callers, 'SV_LEN': all_lengths, 'BREAKDANCER': breakdancer, 'PINDEL': pindel, 'LUMPY': lumpy, 'MANTA': manta, 'DELLY': delly, 'GRIDSS': gridss}
    reformat_frame = pd.DataFrame(all_reformat)
    return reformat_frame

def overlap_samples(all_samples, over_t):
    all_samples['SAMPLE_COUNTS'] = ''
    over_t = int(over_t)
    for r1 in all_samples.itertuples():
        all_callers = []
        for r2 in all_samples.itertuples():
            if r1.Index != r2.Index:
                if r1.SAMPLE != r2.SAMPLE:
                    if r1.CHROM == r2.CHROM:
                        if float(r2.START) <= float(r1.START) and float(r2.STOP) < float(r1.STOP) and float(r2.STOP) > float(r1.START):
                            overlap = (float(r2.STOP) - float(r1.START)) / (float(r1.STOP) - float(r1.START)) * 100
                            overlap_2 = (float(r2.STOP) - float(r1.START)) / (float(r2.STOP) - float(r2.START)) * 100
                            if overlap >= over_t and overlap_2 >= over_t:
                                if r1.SAMPLE_COUNTS == '' and r2.SAMPLE_COUNTS == '':
                                    new_sample_info = str(r1.SAMPLE) + ':' + str(r2.SAMPLE)
                                elif r1.SAMPLE_COUNTS == '' and r2.SAMPLE_COUNTS != '':
                                    new_sample_info = str(r1.SAMPLE) + ':' + str(r2.SAMPLE_COUNTS)
                                elif r1.SAMPLE_COUNTS != '' and r2.SAMPLE_COUNTS == '':
                                    new_sample_info = str(r1.SAMPLE_COUNTS) + ':' + str(r2.SAMPLE)
                                else:
                                    info1 = r1.SAMPLE_COUNTS.split(':')
                                    info2 = r2.SAMPLE_COUNTS.split(':')
                                    new_info = info1 + list(set(info2) - set(info1))
                                    new_sample_info = ':'
                                    new_sample_info = new_sample_info.join(new_info)

                                new_b = r1.BREAKDANCER + ';' + r2.BREAKDANCER
                                new_p = r1.PINDEL + ';' + r2.PINDEL
                                new_l = r1.LUMPY + ';' + r2.LUMPY
                                new_m = r1.MANTA + ';' + r2.MANTA
                                new_d = r1.DELLY + ';' + r2.DELLY
                                new_g = r1.GRIDSS + ';' + r2.GRIDSS
                                new_length = float(r1.STOP) - float(r2.START)
                                all_samples.at[r1.Index, 'START'] = r2.START
                                all_samples.at[r1.Index, 'SAMPLE'] = r1.SAMPLE
                                all_samples.at[r1.Index, 'SAMPLE_COUNTS'] = new_sample_info
                                all_samples.at[r1.Index, 'STOP'] = r1.STOP
                                all_samples.at[r1.Index, 'CHROM'] = r1.CHROM
                                all_samples.at[r1.Index, 'SV_LEN'] = new_length
                                all_samples.at[r1.Index, 'NUM_CALLERS'] = r1.NUM_CALLERS
                                all_samples.at[r1.Index, 'BREAKDANCER'] = new_b
                                all_samples.at[r1.Index, 'PINDEL'] = new_p
                                all_samples.at[r1.Index, 'LUMPY'] = new_l
                                all_samples.at[r1.Index, 'MANTA'] = new_m
                                all_samples.at[r1.Index, 'DELLY'] = new_d
                                all_samples.at[r1.Index, 'GRIDSS'] = new_d
                                all_samples.drop(r2.Index, inplace=True)

                        elif float(r2.START) > float(r1.START) and float(r2.STOP) >= float(r1.STOP) and float(r2.START) < float(r1.STOP):
                            overlap = (float(r1.STOP) - float(r2.START)) / (float(r1.STOP) - float(r1.START)) * 100
                            overlap_2 = (float(r1.STOP) - float(r2.START)) / (float(r2.STOP) - float(r2.START)) * 100
                            if overlap >= over_t and overlap_2 >= over_t:
                                if r1.SAMPLE_COUNTS == '' and r2.SAMPLE_COUNTS == '':
                                    new_sample_info = str(r1.SAMPLE) + ':' + str(r2.SAMPLE)
                                elif r1.SAMPLE_COUNTS == '' and r2.SAMPLE_COUNTS != '':
                                    new_sample_info = str(r1.SAMPLE) + ':' + str(r2.SAMPLE_COUNTS)
                                elif r1.SAMPLE_COUNTS != '' and r2.SAMPLE_COUNTS == '':
                                    new_sample_info = str(r1.SAMPLE_COUNTS) + ':' + str(r2.SAMPLE)
                                else:
                                    info1 = r1.SAMPLE_COUNTS.split(':')
                                    info2 = r2.SAMPLE_COUNTS.split(':')
                                    new_info = info1 + list(set(info2) - set(info1))
                                    new_sample_info = ':'
                                    new_sample_info = new_sample_info.join(new_info)
                                new_length = float(r2.STOP) - float(r1.START)
                                new_b = r1.BREAKDANCER + ';' + r2.BREAKDANCER
                                new_p = r1.PINDEL + ';' + r2.PINDEL
                                new_l = r1.LUMPY + ';' + r2.LUMPY
                                new_m = r1.MANTA + ';' + r2.MANTA
                                new_d = r1.DELLY + ';' + r2.DELLY
                                new_g = r1.GRIDSS + ';' + r2.GRIDSS
                                all_samples.at[r1.Index, 'SAMPLE_COUNTS'] = new_sample_info
                                all_samples.at[r1.Index, 'CHROM'] = r1.CHROM
                                all_samples.at[r1.Index, 'SV_LEN'] = new_length
                                all_samples.at[r1.Index, 'NUM_CALLERS'] = r1.NUM_CALLERS
                                all_samples.at[r1.Index, 'STOP'] = r2.STOP
                                all_samples.at[r1.Index, 'START'] = r1.START
                                all_samples.at[r1.Index, 'SAMPLE'] = r1.SAMPLE
                                all_samples.at[r1.Index, 'BREAKDANCER'] = new_b
                                all_samples.at[r1.Index, 'PINDEL'] = new_p
                                all_samples.at[r1.Index, 'LUMPY'] = new_l
                                all_samples.at[r1.Index, 'MANTA'] = new_m
                                all_samples.at[r1.Index, 'DELLY'] = new_d
                                all_samples.at[r1.Index, 'GRIDSS'] = new_d
                                all_samples.drop(r2.Index, inplace=True)

                        elif float(r2.START) <= float(r1.START) and float(r2.STOP) >= float(r1.STOP):

                            overlap = (float(r1.STOP) - float(r1.START)) / (float(r2.STOP) - float(r2.START)) * 100
                            if overlap >= over_t:
                                if r1.SAMPLE_COUNTS == '' and r2.SAMPLE_COUNTS == '':
                                    new_sample_info = str(r1.SAMPLE) + ':' +  str(r2.SAMPLE)
                                elif r1.SAMPLE_COUNTS == '' and r2.SAMPLE_COUNTS != '':
                                    new_sample_info = str(r1.SAMPLE) + ':' + str(r2.SAMPLE_COUNTS)
                                elif r1.SAMPLE_COUNTS != '' and r2.SAMPLE_COUNTS == '':
                                    new_sample_info = str(r1.SAMPLE_COUNTS) + ':' +  str(r2.SAMPLE)
                                else:
                                    info1 = r1.SAMPLE_COUNTS.split(':')
                                    info2 = r2.SAMPLE_COUNTS.split(':')
                                    new_info = info1 + list(set(info2) - set(info1))
                                    new_sample_info = ':'
                                    new_sample_info = new_sample_info.join(new_info)
                                new_length = float(r2.STOP) - float(r2.START)
                                new_b = r1.BREAKDANCER + ';' + r2.BREAKDANCER
                                new_p = r1.PINDEL + ';' + r2.PINDEL
                                new_l = r1.LUMPY + ';' + r2.LUMPY
                                new_m = r1.MANTA + ';' + r2.MANTA
                                new_d = r1.DELLY + ';' + r2.DELLY
                                new_g = r1.GRIDSS + ';' + r2.GRIDSS
                                all_samples.at[r1.Index, 'SAMPLE_COUNTS'] = new_sample_info
                                all_samples.at[r1.Index, 'START'] = r2.START
                                all_samples.at[r1.Index, 'STOP'] = r2.STOP
                                all_samples.at[r1.Index, 'CHROM'] = r1.CHROM
                                all_samples.at[r1.Index, 'SV_LEN'] = new_length
                                all_samples.at[r1.Index, 'NUM_CALLERS'] = r1.NUM_CALLERS
                                all_samples.at[r1.Index, 'SAMPLE'] = r1.SAMPLE
                                all_samples.at[r1.Index, 'BREAKDANCER'] = new_b
                                all_samples.at[r1.Index, 'PINDEL'] = new_p
                                all_samples.at[r1.Index, 'LUMPY'] = new_l
                                all_samples.at[r1.Index, 'MANTA'] = new_m
                                all_samples.at[r1.Index, 'DELLY'] = new_d
                                all_samples.at[r1.Index, 'GRIDSS'] = new_d
                                all_samples.drop(r2.Index, inplace=True)


                        elif float(r2.START) >= float(r1.START) and float(r2.STOP) <= float(r1.STOP):

                            overlap = (float(r2.STOP) - float(r2.START)) / (float(r1.STOP) - float(r1.START)) * 100
                            if overlap >= over_t:
                                if r1.SAMPLE_COUNTS == '' and r2.SAMPLE_COUNTS == '':
                                    new_sample_info = str(r1.SAMPLE) + ':' +  str(r2.SAMPLE)
                                elif r1.SAMPLE_COUNTS == '' and r2.SAMPLE_COUNTS != '':
                                    new_sample_info = str(r1.SAMPLE) + ':' + str(r2.SAMPLE_COUNTS)
                                elif r1.SAMPLE_COUNTS != '' and r2.SAMPLE_COUNTS == '':
                                    new_sample_info = str(r1.SAMPLE_COUNTS) + ':' + str(r2.SAMPLE)
                                else:
                                    info1 = r1.SAMPLE_COUNTS.split(':')
                                    info2 = r2.SAMPLE_COUNTS.split(':')
                                    new_info = info1 + list(set(info2) - set(info1))
                                    new_sample_info = ':'
                                    new_sample_info = new_sample_info.join(new_info)
                                new_b = r1.BREAKDANCER + ';' + r2.BREAKDANCER
                                new_p = r1.PINDEL + ';' + r2.PINDEL
                                new_l = r1.LUMPY + ';' + r2.LUMPY
                                new_m = r1.MANTA + ';' + r2.MANTA
                                new_d = r1.DELLY + ';' + r2.DELLY
                                new_g = r1.GRIDSS + ';' + r2.GRIDSS
                                all_samples.at[r1.Index, 'SAMPLE_COUNTS'] = new_sample_info
                                all_samples.at[r1.Index, 'START'] = r1.START
                                all_samples.at[r1.Index, 'STOP'] = r1.STOP
                                all_samples.at[r1.Index, 'CHROM'] = r1.CHROM
                                all_samples.at[r1.Index, 'SV_LEN'] = r1.SV_LEN
                                all_samples.at[r1.Index, 'NUM_CALLERS'] = r1.NUM_CALLERS
                                all_samples.at[r1.Index, 'SAMPLE'] = r1.SAMPLE
                                all_samples.at[r1.Index, 'BREAKDANCER'] = new_b
                                all_samples.at[r1.Index, 'PINDEL'] = new_p
                                all_samples.at[r1.Index, 'LUMPY'] = new_l
                                all_samples.at[r1.Index, 'MANTA'] = new_m
                                all_samples.at[r1.Index, 'DELLY'] = new_d
                                all_samples.at[r1.Index, 'GRIDSS'] = new_d
                                all_samples.drop(r2.Index, inplace=True)

    all_samples
    how_many = []
    dedup_samples = []
    for row in all_samples.itertuples():
        if row.SAMPLE_COUNTS == '':
            how_many.append(1)
            dedup_samples.append(str(row.SAMPLE))
        else:
            called_sams = row.SAMPLE_COUNTS.split(':')
            called_sams = list(dict.fromkeys(called_sams))
            total = len(called_sams)
            how_many.append(total)
            the_samples = ':'
            the_samples = the_samples.join(called_sams)
            dedup_samples.append(the_samples)


    all_samples['SAMPLE_COUNTS'] = dedup_samples
    all_samples['NUM_SAMPLES'] = how_many
    all_samples.sort_values(by='CHROM', inplace=True)
    return all_samples

In [ ]:
cd /path/to/data
mkdir -p allSV 
mkdir -p mergeSV

In [ ]:
#reformats output from files to make compatible 

pathToDir = "path/to/data/"

for num in samples:
    fName = pathToDir + "breakdancer/" + 'AS' + str(num) + '_breakdancer.txt'
    outfile = pathToDir + 'AS' + str(num) + ".reformat.vcf"
    reformat = reformat_breakdancer(fName)
    reformat.to_csv(outfile, sep='\t',index=False)
    
    break_file = pathToDir + "breakdancer/" + "AS" + num + '.reformat.vcf'
    breakdancer = read_vcf(break_file)
    reformat_b = reformat_ss(breakdancer, 'breakdancer')
    output_fileb = 'pathToDirbreakdancer/' + "AS" + num + '_breakdancer.ss.vcf'
    reformat_b.to_csv(output_fileb, sep='\t', index=False)
    
    manta_file = pathToDir + 'manta/' + "AS" + num + '_manta.vcf'
    manta = read_vcf(manta_file)
    reformat_m = reformat_ss(manta, 'manta')
    output_filem = pathToDir + 'manta/' + "AS" + num + '_manta.ss.vcf'
    reformat_m.to_csv(output_filem, sep='\t', index=False)
    
    delly_file = pathToDir + 'delly/' + "AS" + num + '.vcf'
    delly = read_vcf(delly_file)
    reformat_d = reformat_ss(delly, 'delly')
    output_filed = pathToDir 'delly/' + "AS" + num + '_delly.ss.vcf'
    reformat_d.to_csv(output_filed, sep='\t', index=False)
    
    pindel_file = pathToDir + 'pindel/' + "AS" + num + '_pindel.vcf'
    pindel = read_vcf(pindel_file)
    reformat_p = reformat_ss(pindel, 'pindel')
    output_filep = pathToDir + 'pindel/' + "AS" + num + '_pindel.ss.vcf'
    reformat_p.to_csv(output_filep, sep='\t', index=False)
    
    gridss_file = pathToDir + 'gridss/' + "AS" + num + '_gridss.ann.bed'
    gridss = pd.read_csv(gridss_file, sep='\t', names=['CHROM', 'START_POS', 'STOP_POS', 'SV_TYPE', 'QUAL', 'CALLER'])
    gridss['CALLER'] = 'gridss'
    output_fileg = pathToDir + 'gridss/' + "AS" + num + '_gridss.ss.vcf'
    gridss.to_csv(output_fileg, sep='\t', index=False)
    
    lumpy_file = pathToDir + 'lumpy/' + "AS" + num + '_lumpy.ann.bed'
    lumpy = pd.read_csv(lumpy_file, sep='\t', names=['CHROM', 'START_POS', 'STOP_POS', 'SV_TYPE', 'QUAL', 'CALLER'])
    lumpy['CALLER'] = 'lumpy'
    output_filel = pathToDir + 'lumpy/' + "AS" + num + '_lumpy.ss.vcf'
    lumpy.to_csv(output_filel, sep='\t', index=False)
 
    combined_table = combine_SV(output_fileb,output_filep,output_filed,output_filem,output_filel,output_fileg)
    out_file = 'pathToDirallSV/' + 'AS' + num + '_allSV.txt'
    combined_table.to_csv(out_file, sep='\t',index=False)

In [ ]:
#takes all concat SV files frome each sample and overlap the calls to make a merged file SV

path = "/path/to/data/allSV/"

#INSERT LIST OF SAMPLES
##sams = ["149", "524", "525", "542", "553","581", "591", "610", "625", "626", "647", "697", "717"]
for num in sams:
    file = path + 'AS' + num  + '_allSV.txt'
    outfile = path + 'AS' + num  + '_mergeSV_all.txt'
    sample_table = make_merged(file,10000000000,1,75,3)
    new_sample_table = dedup_reformat(sample_table, 70)
    new_sample_table.to_csv(outfile, sep='\t')


#reformats first column of each sample
for num in sams:
    samDf = path + 'AS' + num + '_mergeSV_all.txt'
    samFrame = pd.read_csv(samDf, sep='\t')
    samFrame.rename(columns={'Unnamed: 0': 'SAMPLE'}, inplace=True)
    samID = 'AS' + num
    samFrame['SAMPLE'] = samID
    outfile = path + 'AS' + num  + '_mergeSV_all.txt'
    samFrame.to_csv(outfile, sep='\t', index=False)

In [ ]:
#reads in each samples merged calls and and then overlaps between samples 
###FIGURE OUT BETTER WAY OF DOING THIS
bm149 = pd.read_csv('/path/to/data/allSV/bm149_mergeSV_all.txt', sep='\t')
bm525 = pd.read_csv('/path/to/data/allSV/bm525_mergeSV_all.txt', sep='\t')
bm542 = pd.read_csv('/path/to/data/allSV/bm542_mergeSV_all.txt', sep='\t')
bm553 = pd.read_csv('/path/to/data/allSV/bm553_mergeSV_all.txt', sep='\t')
bm581 = pd.read_csv('/path/to/data/allSV/bm581_mergeSV_all.txt', sep='\t')
bm591 = pd.read_csv('/path/to/data/allSV/bm591_mergeSV_all.txt', sep='\t')
bm610 = pd.read_csv('/path/to/data/allSV/bm610_mergeSV_all.txt', sep='\t')
bm625 = pd.read_csv('/path/to/data/allSV/bm625_mergeSV_all.txt', sep='\t')
bm626 = pd.read_csv('/path/to/data/allSV/bm626_mergeSV_all.txt', sep='\t')
bm647 = pd.read_csv('/path/to/data/allSV/bm647_mergeSV_all.txt', sep='\t')
bm697 = pd.read_csv('/path/to/data/allSV/bm697_mergeSV_all.txt', sep='\t')
bm717 = pd.read_csv('/path/to/data/allSV/bm717_mergeSV_all.txt', sep='\t')

###FIX
all_samples = pd.concat([bm149,bm524, bm525, bm542, bm553, bm581, bm591, bm610, bm625, bm626, bm647, bm697, bm717], axis=0, ignore_index=True)

#overlaps at 75% to consider the "same"
all_samples = overlap_samples(all_samples, 75)
all_samples.sort_values(by=['CHROM','START'], inplace=True)

#Removes any SV that all samples share since indicates ref 
remove = all_samples.loc[all_samples['NUM_SAMPLES'] < 13]
remove.to_csv("/path/to/data/mergeSV/noShareSV.txt", sep="\t", index=False)
bed_format = remove.filter(['CHROM','START','STOP'], axis=1)
bed_format.to_csv('/path/to/data/mergeSV/noShareSV.bed', sep='\t', index=False, header=False)

In [ ]:
cd /path/to/data/mergeSV
mkdir -p finalSV
bedtools igv -slop 10 -path /path/to/data/mergeSV -i /path/to/data/mergeSV/noShareSV.bed > /path/to/data/finalSV/otherScript.sh

In [ ]:
cd /path/to/data/mergeSV/finalSV
for photo in *.png ; do convert $photo -rotate -90 $photo ; done

SNP/INDEL calling 

In [ ]:
#John Stuff

Phylogeny